In [ ]:
user = 'puzzlelists@gmail.com'
password = 'gggoahqntwxenpbe'
imap_url = 'imap.gmail.com'

In [ ]:
# Importing libraries
import imaplib, email
 
user = 'puzzlelists@gmail.com'
password = 'gggoahqntwxenpbe'
imap_url = 'pop.gmail.com'
 
# Function to get email content part i.e its body part
def get_body(msg):
    if msg.is_multipart():
        return get_body(msg.get_payload(0))
    else:
        return msg.get_payload(None, True)
 
# Function to search for a key value pair
def search(key, value, con):
    result, data = con.search(None, key, '"{}"'.format(value))
    return data
 
# Function to get the list of emails under this label
def get_emails(result_bytes):
    msgs = [] # all the email data are pushed inside an array
    for num in result_bytes[0].split():
        typ, data = con.fetch(num, '(RFC822)')
        msgs.append(data)
 
    return msgs
 
# this is done to make SSL connection with GMAIL
con = imaplib.IMAP4_SSL(imap_url)
 
# logging the user in
con.login(user, password)
 
# calling function to check for email under this label
con.select('Inbox')
 
 # fetching emails from this user "tu**h*****1@gmail.com"
msgs = get_emails(search('FROM', 'MY_ANOTHER_GMAIL_ADDRESS', con))
 
# Uncomment this to see what actually comes as data
print(msgs)
 
 
# Finding the required content from our msgs
# User can make custom changes in this part to
# fetch the required content he / she needs
 
# printing them by the order they are displayed in your gmail
for msg in msgs[::-1]:
    for sent in msg:
        if type(sent) is tuple:
 
            # encoding set as utf-8
            content = str(sent[1], 'utf-8')
            data = str(content)
 
            # Handling errors related to unicodenecode
            try:
                indexstart = data.find("ltr")
                data2 = data[indexstart + 5: len(data)]
                indexend = data2.find("</div>")
 
                # printing the required content which we need
                # to extract from our email i.e our body
                print(data2[0: indexend])
 
            except UnicodeEncodeError as e:
                pass

In [ ]:
msgs

In [ ]:
import poplib
import string, random
import logging
from io import StringIO

SERVER = "smtp.gmail.com"
USER  = user
PASSWORD = password

# connect to server
logging.debug('connecting to ' + SERVER)
server = poplib.POP3_SSL(SERVER)
#server = poplib.POP3(SERVER)

# log in
logging.debug('log in')
server.user(USER)
server.pass_(PASSWORD)

In [ ]:
# connect to server
logging.debug('connecting to ' + SERVER)
server = poplib.POP3_SSL(SERVER)
#server = poplib.POP3(SERVER)

# login
logging.debug('logging in')
server.user(USER)
server.pass_(PASSWORD)

# list items on server
logging.debug('listing emails')
resp, items, octets = server.list()

# download the first message in the list
id, size = items[0].split()
resp, text, octets = server.retr(id)

# convert list to Message object
text = str.join(text, "\n")
file = StringIO.StringIO(text)
message = rfc822.Message(file)

# output message
print(message['From']),
print(message['Subject']),
print(message['Date']),
print(message.fp.read())   

In [ ]:
# connect to server
logging.debug('connecting to ' + SERVER)
server = poplib.POP3_SSL(SERVER)
#server = poplib.POP3(SERVER)

# login
logging.debug('logging in')
server.user(USER)
server.pass_(PASSWORD)

In [ ]:
# connect to server
logging.debug('connecting to ' + SERVER)
server = poplib.POP3_SSL(SERVER)
#server = poplib.POP3(SERVER)

# login
logging.debug('logging in')
server.user(USER)
server.pass_(PASSWORD)

# list items on server
logging.debug('listing emails')
resp, items, octets = server.list()

In [ ]:
resp

In [ ]:
items

In [ ]:
octets

In [ ]:
# download the first message in the list
items[0]

In [ ]:

# download the first message in the list
id, size = items[0].split()

In [ ]:
server.retr(id)

In [ ]:
import imaplib
import email
from email.header import decode_header
import webbrowser
import os

# account credentials
username = "krishnaraj.kpt@outlook.com"
password = "BBQtJTSs8uQh57aw"
# use your email provider's IMAP server, you can look for your provider's IMAP server on Google
# or check this page: https://www.systoolsgroup.com/imap/
# for office 365, it's this:
imap_server = "outlook.office365.com"

def clean(text):
    # clean text for creating a folder
    return "".join(c if c.isalnum() else "_" for c in text)
# create an IMAP4 class with SSL 
imap = imaplib.IMAP4_SSL(imap_server)
# authenticate
imap.login(username, password)
status, messages = imap.select("INBOX")
# number of top emails to fetch
N = 850
# total number of emails
messages = int(messages[0])
print(messages)
for i in range(messages, messages-N, -1):
    # fetch the email message by ID
    res, msg = imap.fetch(str(i), "(RFC822)")
    for response in msg:
        try: 
            if isinstance(response, tuple):
                # parse a bytes email into a message object
                msg = email.message_from_bytes(response[1])
                # decode the email subject
                subject, encoding = decode_header(msg["Subject"])[0]
                if isinstance(subject, bytes):
                    # if it's a bytes, decode to str
                    subject = subject.decode(encoding)
                # decode email sender
                From, encoding = decode_header(msg.get("From"))[0]
                print(encoding)
                if isinstance(From, bytes):
                    From = From.decode(encoding)
                print("Subject:", subject)
                print("From:", From)
                # if the email message is multipart
                if msg.is_multipart():
                    # iterate over email parts
                    for part in msg.walk():
                        # print("testing", part.as_string())
                        # extract content type of email
                        content_type = part.get_content_type()
                        content_disposition = str(part.get("Content-Disposition"))
                        try:
                            # get the email body
                            body = part.get_payload(decode=True).decode()
                        except:
                            pass
                        if content_type == "text/plain" and "attachment" not in content_disposition:
                            # print text/plain emails and skip attachments
                            print(body)
                        elif "attachment" in content_disposition:
                            # download attachment
                            filename = part.get_filename()
                            if filename:
                                folder_name = clean(subject)
                                if not os.path.isdir(folder_name):
                                    # make a folder for this email (named after the subject)
                                    os.mkdir(folder_name)
                                filepath = os.path.join(folder_name, filename)
                                # download attachment and save it
                                open(filepath, "wb").write(part.get_payload(decode=True))
                else:
                    # extract content type of email
                    content_type = msg.get_content_type()
                    # get the email body
                    body = msg.get_payload(decode=True).decode()
                    if content_type == "text/plain":
                        # print only text email parts
                        print(body)
                
                print("="*100)
        except:
            pass
# close the connection and logout
imap.close()
imap.logout()

In [ ]:
import imaplib
import email
from email.header import decode_header
import webbrowser
import os

# account credentials
username = "puzzlelists@gmail.com"
password = "gggoahqntwxenpbe"
# use your email provider's IMAP server, you can look for your provider's IMAP server on Google
# or check this page: https://www.systoolsgroup.com/imap/
# for office 365, it's this:
imap_server = "smtp.gmail.com"

def clean(text):
    # clean text for creating a folder
    return "".join(c if c.isalnum() else "_" for c in text)
# create an IMAP4 class with SSL 
imap = imaplib.IMAP4_SSL(imap_server)
# authenticate
imap.login(username, password)
status, messages = imap.select("INBOX")
# number of top emails to fetch
N = 1
# total number of emails
messages = int(messages[0])
for i in range(messages, messages-N, -1):
    # fetch the email message by ID
    res, msg = imap.fetch(str(i), "(RFC822)")
    for response in msg:
        if isinstance(response, tuple):
            # parse a bytes email into a message object
            msg = email.message_from_bytes(response[1])
            # decode the email subject
            subject, encoding = decode_header(msg["Subject"])[0]
            if isinstance(subject, bytes):
                # if it's a bytes, decode to str
                subject = subject.decode(encoding)
            # decode email sender
            From, encoding = decode_header(msg.get("From"))[0]
            print(encoding)
            if isinstance(From, bytes):
                From = From.decode(encoding)
            print("Subject:", subject)
            print("From:", From)
            # if the email message is multipart
            if msg.is_multipart():
                # iterate over email parts
                for part in msg.walk():
                    print("testing", part.as_string())

In [ ]:
import imaplib
import email
from email.header import decode_header
import os
import csv

def clean(text):
    # clean text for creating a folder
    return "".join(c if c.isalnum() else "_" for c in text)

imap = imaplib.IMAP4_SSL(imap_server)

# authenticate
imap.login(username, password)
status, messages = imap.select("INBOX")

# number of top emails to fetch
N = 850

# total number of emails
messages = int(messages[0])

# CSV file setup
csv_filename = "email_details.csv"
csv_columns = ["Subject", "From", "Content"]
csv_rows = []

for i in range(messages, messages - N, -1):
    # fetch the email message by ID
    res, msg = imap.fetch(str(i), "(RFC822)")
    for response in msg:
        try:
            if isinstance(response, tuple):
                # parse a bytes email into a message object
                msg = email.message_from_bytes(response[1])

                # decode the email subject
                subject, encoding = decode_header(msg["Subject"])[0]
                if isinstance(subject, bytes):
                    # if it's bytes, decode to str
                    subject = subject.decode(encoding)

                # decode email sender
                From, encoding = decode_header(msg.get("From"))[0]
                if isinstance(From, bytes):
                    From = From.decode(encoding)

                # if the email message is multipart
                if msg.is_multipart():
                    # iterate over email parts
                    for part in msg.walk():
                        content_type = part.get_content_type()
                        content_disposition = str(part.get("Content-Disposition"))

                        try:
                            # get the email body
                            body = part.get_payload(decode=True).decode()
                        except:
                            pass

                        # Append details to CSV rows
                        csv_rows.append({"Subject": subject, "From": From, "Content": body})

                else:
                    # extract content type of email
                    content_type = msg.get_content_type()
                    # get the email body
                    body = msg.get_payload(decode=True).decode()

                    # Append details to CSV rows
                    csv_rows.append({"Subject": subject, "From": From, "Content": body})

        except Exception as e:
            print(f"Error processing email: {e}")

# close the connection and logout
imap.close()
imap.logout()

# Write CSV file
with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()
    for row in csv_rows:
        writer.writerow(row)

print(f"Email details exported to {csv_filename}")


In [12]:
import imaplib
import email
import csv

# connect to the IMAP server
mail = imaplib.IMAP4_SSL('imap-mail.outlook.com')
mail.login('krishnaraj.kpt@outlook.com', "BBQtJTSs8uQh57aw")
mail.select('inbox')

# search for all emails
status, data = mail.search(None, 'ALL')
mail_ids = data[0].split()

# create CSV files for headers and information
with open('headers.csv', 'w', newline='') as headers_file, open('information.csv', 'w', newline='') as info_file:
    headers_writer = csv.writer(headers_file)
    info_writer = csv.writer(info_file)

    # write headers to CSV files
    headers_writer.writerow(['Message ID', 'Date', 'From', 'To', 'Subject'])
    info_writer.writerow(['Message ID', 'Date', 'From', 'To', 'Subject'])

    # iterate over all emails
    for mail_id in mail_ids:
        # fetch the email
        status, data = mail.fetch(mail_id, '(RFC822)')
        raw_email = data[0][1]

        # parse the email
        email_message = email.message_from_bytes(raw_email)

        # extract headers and information
        message_id = email_message['Message-ID']
        date = email_message['Date']
        sender = email_message['From']
        recipient = email_message['To']
        subject = email_message['Subject']

        # write headers and information to CSV files
        headers_writer.writerow([message_id, date, sender, recipient, subject])
        info_writer.writerow([message_id, date, sender, recipient, subject])

# close the connection to the IMAP server
mail.close()
mail.logout()

('BYE', [b'Microsoft Exchange Server IMAP4 server signing off.'])